# EDP Elípticas con Diferencias Finitas

Recordemos que una ecuación diferencial parcial o EDP (PDE en inglés) es una ecuación que involucra funciones en dos o más variables y sus derivadas parciales. En este caso estudiamos las ecuaciones de tipo

$$Au_{xx} + Bu_{xy} + Cu_{yy} + f(u_x, u_y, u, x, y) = 0$$

donde $A$, $B$ y $C$ son escalares y además $x,y$ son las variables independientes.

El discriminante de estas ecuaciones $B^2 -4AC$ nos indicará si una ecuación es parabólica, elíptica o hiperbólica. Las ecuaciones que nos interesan ahora son elípticas, por lo tanto cumplen con que $B^2-4AC < 0$.

Las ecuaciones elípticas, a diferencia de los otros tipos, poseen condiciones de borde para ambas variables independientes.

Definiciones:

1. Nunca deben olvidar la fórmula del **laplaciano** de una función! Es simplemente la suma de las segundas derivadas respecto a una misma variable.
$$\mathcal{L}(u) = \Delta u = u_{xx} + u_{yy}$$

2. La ecuación $\Delta u = f(x,y)$ es conocida como **Ecuación de Poisson**. En particular cuando $f(x,y) = 0$ la ecuación se conoce como **Ecuación de Laplace**. (Notar cómo la definición de las ecuaciones es consistente con que sean elípticas...)

Existen dos tipos de condiciones de borde. Se puede imponer una condición en el borde tanto para $u$ (condiciones de Dirichlet) como para alguna derivada direccional $\partial u / \partial n$ (Condiciones de Neumann)

## Aplicación del Método

Como siempre declaramos el problema: Resolveremos la ecuación $\Delta u = f$ en un rectángulo dado $[x_l, x_r] \times [y_b, y_t]$. Además consideremos las condiciones de borde de Dirichlet que definen alguna función $g$ para cada borde, digamos:

\begin{align*}
u(x,y_b) = g_1(x)\\
u(x,y_t) = g_2(x)\\
u(x_l,y) = g_3(y)\\
u(x_r,y) = g_4(y)
\end{align*}

Ahora debemos discretizar el dominio bidimensional. Para $m$ puntos en el eje horizontal y $n$ en el vertical, es decir con $M = m-1$ y $N = n-1$ steps de tamaño $h = (x_r − x_l)/M$ and $k=(y_t − y_ b)/N$. Si reemplazamos las diferencias finitas centradas que estudiamos anteriormente en la ecuación de Poisson obtenemos:

$$\frac{u(x-h,y) -2u(x,y) + u(x+h,y)}{h^2} + \mathcal{O}(h^2) + \frac{u(x,y-k) -2u(x,y) + u(x,y+k)}{k^2} + \mathcal{O}(k^2)  = 0$$